# Captain Analysis

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import json
import glob
import re

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

from battleship.run_captain_benchmarks import rebuild_captain_summary_from_results
from battleship.utils import resolve_project_path
from battleship.agents import EIGCalculator, CodeQuestion, Question
from battleship.game import Board

from analysis import CAPTAIN_TYPE_LABELS
from analysis import human_round_summaries

In [ ]:
%config InlineBackend.figure_format = 'retina'

# set seaborn color palette
sns.set_palette("Set2")

# set seaborn style
sns.set_style("whitegrid")
sns.set_context("talk")

In [ ]:
HUMAN_EXPERIMENT_NAME = "battleship-final-data"
PATH_DATA = os.path.join("data", HUMAN_EXPERIMENT_NAME)
PATH_EXPORT = os.path.join(PATH_DATA, "export")

CAPTAIN_EXPERIMENT_PATH = (
    "experiments/collaborative/captain_benchmarks/"
)

## Data loading

### Human data

In [ ]:
human_df = human_round_summaries(
    experiment_path=PATH_DATA,
)
human_df = pd.DataFrame(human_df)

human_df = human_df.assign(llm="human")
human_df

### Model data

In [ ]:
model_round_data_unresolved_paths = [
    ("gpt-4o", "run_2025_08_25_16_28_19"),
    ("gpt-5", "run_2025_08_25_22_02_29"),
    ("baseline", "run_2025_08_26_17_23_23"),
    ("llama-4-scout", "run_2025_08_26_17_56_46"),
]

model_round_data_paths = [
    (name, resolve_project_path(os.path.join(CAPTAIN_EXPERIMENT_PATH, path)))
    for name, path in model_round_data_unresolved_paths
]
for name, path in model_round_data_paths:
    if not os.path.exists(path):
        print(f"The path {path} does not exist.")

dfs = []
for name, path in model_round_data_paths:
    df = pd.DataFrame(rebuild_captain_summary_from_results(path))
    df["llm"] = name
    dfs.append(df)

model_df = pd.concat(dfs, ignore_index=True)
model_df

In [ ]:
# Append summary_df to round_df
combined_df = pd.concat([human_df, model_df], ignore_index=True)
combined_df

## Precision/Recall Stats

In [ ]:
g = sns.catplot(data=combined_df, kind="box", col="llm", x="captain_type", y="f1_score", hue="captain_type")
g.set_xticklabels(rotation=90)

In [ ]:
sorted_df = combined_df.sort_values(by='captain_type', ascending=True)
# sorted_df = sorted_df[sorted_df['captain_type'].isin(['human', 'RandomCaptain', 'MAPCaptain','LLMDecisionCaptain', 'EIGCaptain', 'MAPEIGCaptain'])]

sns.catplot(data=sorted_df, kind="box", x="captain_type", y="f1_score", hue="llm")

plt.xticks(rotation=90)

In [ ]:
plt.figure(figsize=(10, 6))

for captain_type, group in combined_df.groupby('captain_type'):
    f1_scores = np.sort(group['f1_score'].dropna())
    cdf = np.arange(1, len(f1_scores) + 1) / len(f1_scores)
    plt.step(f1_scores, cdf, where='post', label=captain_type)

plt.xlabel('F1 Score')
plt.ylabel('Cumulative Probability')
plt.title('CDF of F1 Score by Captain Type')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
sns.barplot(data=combined_df, x="captain_type", y="precision", hue="captain_type")
plt.xticks(rotation=45)

In [ ]:
sns.barplot(data=combined_df, x="captain_type", y="recall", hue="captain_type")
plt.xticks(rotation=45)

In [ ]:
combined_df["move_count"] = combined_df["hits"] + combined_df["misses"]

sns.barplot(data=combined_df, x="captain_type", y="move_count", hue="captain_type")
plt.xticks(rotation=45)

## EIG Stats

In [ ]:
# Path to run directory
# base_path = resolve_project_path("experiments/collaborative/captain_benchmarks/run_combined/run_4o_mapeig_cot_captain")
base_path = resolve_project_path(
    "experiments/collaborative/captain_benchmarks/run_combined/run_4o_llmdecision_captain"
)

# Find all captain.json files in subdirectories
captain_files = glob.glob(os.path.join(base_path, '**/captain/captain.json'), recursive=True)

# Dictionary to store eig values by file
eig_values_by_file = {}
# Initialize list to store data for DataFrame
eig_data_list = []

# Extract eig values from each file
for file_path in captain_files:
    # Get relative path for naming
    rel_path = os.path.relpath(file_path, base_path)

    # Extract round_id from path
    # Use regex to extract the part after 'round_' in the relative path
    match = re.search(r'round_([a-zA-Z0-9]+)', rel_path)
    round_id = match.group(1) if match else None

    with open(file_path, 'r') as f:
        data = json.load(f)

    # Extract eig values, skipping None/null values
    for idx, datum in enumerate(data):
        if 'eig' in datum and datum['eig'] is not None and 'question' in datum and datum['question'] is not None:
            question_text = datum['question']['question']['text'] if datum['question']['question'] and 'text' in datum['question']['question'] else "No question text"
            eig_value = datum['eig']

            eig_questions = datum.get("eig_questions", [])


            if eig_questions is not None:
                if len(eig_questions) != 0:
                    eig_questions = [(q['question']['question']['text'],q['eig'], None) for q in eig_questions]
                    max_eig = max([eq[1] for eq in eig_questions if eq[1] is not None])
                    eig_questions = [(q[0], q[1], q[1] == max_eig) for q in eig_questions]

            # Add to data list
            eig_data_list.append({
                'round_id': round_id,
                'question_idx': idx,
                'question': question_text,
                'eig': eig_value,
                'eig_questions': eig_questions,
            })

# Create DataFrame from the list
model_eig_df = pd.DataFrame(eig_data_list)

model_eig_df

In [ ]:
model_eig_df["eig"].hist()

In [ ]:
# /////////////////////////////////////////////////
# This cell calculates EIG for human questions (and saves it to notebooks/human_eig_df.csv)
# Caution: This will take 1-2 mins to run if human_eig_df.csv doesn't exist in the notebooks directory
# \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\

# JSON file to pull the code translations of human questions from
input_json_path = resolve_project_path("experiments/collaborative/spotter_benchmarks/o4-mini_CodeSpotterModel_True.json")

def extract_questions_and_boards_to_dataframe(json_path):
    """
    Extracts all questions asked and the board state at the time they were asked from a JSON file
    and returns the data as a pandas DataFrame.

    Args:
        json_path (str): Path to the input JSON file.

    Returns:
        pd.DataFrame: A DataFrame containing the extracted questions and board states.
    """
    if not os.path.exists(json_path):
        raise FileNotFoundError(f"The file {json_path} does not exist.")

    with open(json_path, 'r') as f:
        data = json.load(f)

    extracted_data = []

    for entry in data:
        if "question" in entry and "occTiles" in entry:
            question = entry["question"]
            program = entry["program"]
            board_state = entry["occTiles"]
            answer = entry["answer"]
            true_answer = entry["true_answer"]

            if answer.lower() == "true":
                answer = "yes"
            if answer.lower() == "false":
                answer = "no"

            extracted_data.append({
                "question": question,
                "program": program,
                "board_state": board_state,
                "answer": answer,
                "true_answer": true_answer,
                "correct": answer == true_answer
            })

    return pd.DataFrame(extracted_data)


if os.path.exists('human_eig_df.csv'):
    human_eig_df = pd.read_csv('human_eig_df.csv')
else:
    human_eig_df = extract_questions_and_boards_to_dataframe(input_json_path)
    human_eig_df = human_eig_df[human_eig_df['correct'] == True]

    eig_calculator = EIGCalculator(samples=1000, timeout=15, epsilon=0)

    # Add a new column to store EIG values
    human_eig_df["calculated_eig"] = None

    for idx, row in human_eig_df.iterrows():
            # Create a CodeQuestion instance
            code_question = CodeQuestion(
                question=Question(row["question"]),
                fn_text=row["program"],
                translation_prompt="",
                completion={}
            )

            # Convert board_state to a Board instance
            board = Board.from_occ_tiles(row["board_state"])

            # Calculate EIG
            eig_value = eig_calculator(code_question, board)
            human_eig_df.at[idx, "calculated_eig"] = eig_value

    human_eig_df.to_csv('human_eig_df.csv', index=False)

human_eig_df

In [ ]:
import seaborn as sns

import matplotlib.pyplot as plt

# Prepare the data for plotting
plot_data = pd.DataFrame({
    'EIG Values': pd.concat([model_eig_df["eig"], human_eig_df["calculated_eig"]], ignore_index=True),
    'Source': ['model_eig_df'] * len(model_eig_df) + ['human_eig_df'] * len(human_eig_df)
})

# Create a boxplot instead of a scatter plot
sns.boxplot(data=plot_data, x='Source', y='EIG Values', palette='Set2')

# Add labels and title
plt.title('Categorical Scatter Plot of EIG Values')
plt.xlabel('Source')
plt.ylabel('EIG Values')
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

# Calculate and print the average EIG values for both distributions
avg_model_eig = model_eig_df["eig"].mean()
avg_human_eig = human_eig_df["calculated_eig"].astype(float).mean()

print(f"Average EIG for eig_df: {avg_model_eig:.4f}")
print(f"Average EIG for output_df: {avg_human_eig:.4f}")